In [81]:
pip install comet_ml

  Obtaining dependency information for comet_ml from https://files.pythonhosted.org/packages/bc/ff/531ccc3e289180c92c15b6cad9b08a1bf9f32ddb807d00b1e3eb9cf7c7f2/comet_ml-3.35.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 46.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 5.8 MB/s eta 0:00:00
  Obtaining dependency information for dulwich!=0.20.33,>=0.20.6 from https://files.pythonhosted.org/packages/d2/85/8c71f5a227cacea450b4a199aae095946a71341ac9da84d304f9376d7791/dulwich-0.21.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.4/586.4 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 35.8 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.6.2
    Uninstalling websocket-client-1.6.2:
      Successfully uninstalled websocket-client-1.6.2
ERROR: pip's de

In [126]:
import comet_ml
comet_ml.init(project_name="Reconcimiento final")
filepath='Best_model1.hdf5'

In [127]:
#Definimos un nuevo experimento en comet
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,)

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: tensorboard, keras, tensorflow.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/lject/reconcimiento-final/b7c19b7bc0d34cab8f72f94767328ccb



In [128]:
import pandas as pd
import os
import numpy as np
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import Dense,Conv2D, Dropout,Activation,MaxPooling2D,Flatten
from tensorflow.keras import regularizers, Model, layers, models
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.regularizers import l1, l2
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import Sequence
from tensorflow.keras import layers

# Ruta de la carpeta que contiene las imágenes
image_folder = '/kaggle/input/celeba/img_align_celeba/img_align_celeba'

# Ruta del archivo de atributos
attribute_file = '/kaggle/input/celeba/Atributos.txt'

# Lee el archivo de atributos en un DataFrame de Pandas
df_attributes = pd.read_csv(attribute_file, delim_whitespace=True)
#elegimos los atributos que sean de nuestro interes de la imagens 
selected_attributes = ['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Bags_Under_Eyes', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blurry', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Rosy_Cheeks', 'Smiling', 'Wavy_Hair', 'Wearing_Lipstick', 'Young']
labels = df_attributes[selected_attributes].values
# Clase de generador de datos personalizado
class CelebADataGenerator(Sequence):
    def __init__(self, image_paths, attributes, batch_size, image_size, shuffle=False):
        self.image_paths = image_paths
        self.attributes = attributes
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.indices = np.arange(len(image_paths))
        #print(self.indices)

        if self.shuffle:
            np.random.shuffle(self.indices)

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        #print((batch_indices.shape))
        #print((batch_indices))
        X = np.zeros((len(batch_indices), *self.image_size, 3))
        y=[]
        for n in batch_indices:
            y.append(self.attributes[n])
        y = np.array(y)
         # Preprocesamiento de etiquetas: Convierte 1 y -1 a 1 y 0
        y = np.where(y == 1., 1., 0.)

        for i, idx in enumerate(batch_indices):
            image_path = self.image_paths[idx]
            img = load_img(image_path, target_size=self.image_size)
            X[i] = img_to_array(img) / 255.0  # Normaliza los valores de píxeles

        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

# Crear el generador de datos
image_paths = [os.path.join(image_folder, filename) for filename in df_attributes.index]
attributes = df_attributes.values
print(type(image_paths))

#data_generator = CelebADataGenerator(image_paths, labels, batch_size, image_size)


<class 'list'>


In [55]:
#pruba para el generador de datos
print(len(labels))
print((image_paths[0]))
print(len(labels[0]))
#%%
# Ejemplo de cómo utilizar el generador para obtener un lote de datos
batch_x, batch_i = data_generator[0]
print(batch_x.shape)  # Forma del lote de imágenes
print(batch_i.shape)  # Forma del lote de atributos


202599
/kaggle/input/celeba/img_align_celeba/img_align_celeba/000001.jpg
24


NameError: name 'data_generator' is not defined

In [73]:
#dividimos los datos en entrenamiento, prueva y test
total_samples = len(image_paths)
train_size = 0.5
validation_size = 0.25
test_size = 0.25

# Dividir los índices de las imágenes
train_indices = np.random.choice(total_samples, int(total_samples * train_size), replace=False)
print(len(train_indices))
remaining_indices = np.setdiff1d(np.arange(total_samples), train_indices)
validation_indices = np.random.choice(remaining_indices, int(total_samples * validation_size), replace=False)
test_indices = np.setdiff1d(remaining_indices, validation_indices)


101299


In [74]:
#extraemos los datos con los indices ya divididos para formar nuestros conjuntos de datos 
itrain_set=[]
atrain_set=[]
ival_set=[]
aval_set=[]
for n in train_indices:
    itrain_set.append(image_paths[n])
    atrain_set.append(labels[n])
for n in validation_indices:
    ival_set.append(image_paths[n])
    aval_set.append(labels[n])


In [82]:
batch_size = 80
image_size = (178, 218)
# Crear generadores de datos para entrenamiento, validación y prueba
train_generator = CelebADataGenerator(itrain_set, atrain_set, batch_size, image_size)
validation_generator = CelebADataGenerator(ival_set, aval_set, batch_size, image_size)
#test_generator = CelebADataGenerator(image_paths[test_indices], attributes[test_indices], batch_size, image_size)


In [83]:

# Prueba de las formas de los datos  
batch_X, batch_y = train_generator[0]
print(batch_X.shape)  # Forma del lote de imágenes
print(batch_y.shape)  # Forma del lote de atributos
batch_X, batch_y = validation_generator[0]
print(batch_X.shape)  # Forma del lote de imágenes
print(batch_y.shape)  # Forma del lote de atributos

print(batch_X[0].shape)
print(type(batch_y))
print(batch_y.shape)



(80, 178, 218, 3)
(80, 24)
(80, 178, 218, 3)
(80, 24)
(178, 218, 3)
<class 'numpy.ndarray'>
(80, 24)


In [90]:
# Prueba 2 de reddd neuronal (residual)
inputs = keras.Input(shape=(178,218,3))
x = layers.Conv2D(100, (2,2))(inputs)
x = layers.Activation('tanh')(x)

for size in [130, 80, 100, 90, 200]:
    residual = x
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)
    residual = layers.Conv2D(
    size, 1, strides=2, padding="same", use_bias=False)(residual)
    x = layers.add([x, residual])


x = layers.Flatten()(x)
x = layers.Dense(300, activation='relu')(x)
x = layers.Dense(270, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(300, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(230, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(100, activation='relu')(x)
x = layers.Dropout(0.2)(x)

outputs = layers.Dense(len(labels[0]), activation='sigmoid')(x)

model = keras.Model(inputs=inputs,outputs=outputs)
model.summary()
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.compile(loss='binary_crossentropy',
              optimizer='RMSprop',
              metrics=['accuracy'])


Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_15 (InputLayer)       [(None, 178, 218, 3)]        0         []                            
                                                                                                  
 conv2d_78 (Conv2D)          (None, 177, 217, 100)        1300      ['input_15[0][0]']            
                                                                                                  
 activation_108 (Activation  (None, 177, 217, 100)        0         ['conv2d_78[0][0]']           
 )                                                                                                
                                                                                                  
 activation_109 (Activation  (None, 177, 217, 100)        0         ['activation_108[0][0]'

In [12]:
# Prueba 1 de red neuronal 
model = Sequential()
model.add(Conv2D(70, (2, 2), input_shape=(178, 218,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(80, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(65, (4, 4)))
model.add(Activation('relu'))

model.add(Conv2D(65, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(60, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(250))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(150))
model.add(Dense(150))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(230,activation='relu'))
model.add(Dense(150,activation='relu'))
model.add(Dense(len(labels[0]),activation='sigmoid'))
model.summary()
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 178, 218, 70)      910       
                                                                 
 activation (Activation)     (None, 178, 218, 70)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 89, 109, 70)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 107, 80)       50480     
                                                                 
 activation_1 (Activation)   (None, 87, 107, 80)       0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 43, 53, 80)        0         
 g2D)                                                   

In [ ]:
steps=int(len(train_indices)/batch_size)
print(steps)

#%%
model.fit(      train_generator,
                steps_per_epoch=steps,
                epochs=20,
                validation_data=validation_generator,
                verbose=1,
                validation_steps=300,
                callbacks=[checkpoint]
                )


1266
Epoch 1/20
1266/1266 [==============================] - ETA: 0s - loss: 0.3480 - accuracy: 0.0630
Epoch 1: val_loss improved from inf to 0.30255, saving model to Best_model1.hdf5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1266/1266 [==============================] - 1166s 913ms/step - loss: 0.3480 - accuracy: 0.0630 - val_loss: 0.3026 - val_accuracy: 0.0725
Epoch 2/20
1266/1266 [==============================] - ETA: 0s - loss: 0.2953 - accuracy: 0.0691
Epoch 2: val_loss improved from 0.30255 to 0.29085, saving model to Best_model1.hdf5
1266/1266 [==============================] - 1130s 893ms/step - loss: 0.2953 - accuracy: 0.0691 - val_loss: 0.2908 - val_accuracy: 0.0728
Epoch 3/20
1266/1266 [==============================] - ETA: 0s - loss: 0.2808 - accuracy: 0.0693
Epoch 3: val_loss improved from 0.29085 to 0.27452, saving model to Best_model1.hdf5
1266/1266 [==============================] - 1124s 888ms/step - loss: 0.2808 - accuracy: 0.0693 - val_loss: 0.2745 - val_accuracy: 0.0702
Epoch 4/20
1266/1266 [==============================] - ETA: 0s - loss: 0.2715 - accuracy: 0.0696
Epoch 4: val_loss did not improve from 0.27452
1266/1266 [==============================] - 1106s 873ms/step - loss: 0.271

In [1]:
#cargamos nuestro mejor modelo a comeet
experiment.log_model("Beste_model1.hdf5", filepath)
#terminamos el experimento.
experiment.end()

NameError: name 'experiment' is not defined

Red para reconocer mi rostro 

In [129]:
#cargamos el modelo entrenado
model1=keras.models.load_model("/kaggle/input/mejormodel/reconocimientoBestModel.hdf5")

In [130]:

# Quitamos las ultimas capas del modelo
capas_convolucion = model1.layers[:-9]  
# Modelo con solo capas de convolucion 
modelo_convolucion = Model(inputs=model1.input, outputs=capas_convolucion[-1].output)

In [131]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Generamos nuestros datos de entrenamiento
image_size = (178, 218)
batch_size = 5
# Generador para los datos de entrenamiento(realizamos cambios en las imagenes)
Train_datagen = ImageDataGenerator(
    rescale=1./255,          
    rotation_range=20,       
    width_shift_range=0.2,  
    height_shift_range=0.2, 
    horizontal_flip=True,    
    fill_mode='nearest' 
)
# Generador para los datos de validacion (realizamos diferentes cambios en las imagenes)
Val_datagen = ImageDataGenerator(
    rescale=1./255,          
    rotation_range=15,       
    width_shift_range=0.3,  
    height_shift_range=0.3, 
    horizontal_flip=True,    
    fill_mode='nearest' 
)
Train = Train_datagen.flow_from_directory(
    '/kaggle/input/entrenar/Entrenamiento/TrainData',  # Ruta a la carpeta de imágenes
    target_size=image_size,  # tamaño de redimensión
    batch_size=batch_size,   # tamaño del lote
    class_mode='binary',  
    shuffle=True
)
val = Val_datagen.flow_from_directory(
    '/kaggle/input/entrenar/Entrenamiento/ValData',  # Ruta a la carpeta de imágenes
    target_size=image_size,  # Tamaño de redimensión
    batch_size=3,   # Tamaño del lote
    class_mode='binary', 
    shuffle=True
)




Found 84 images belonging to 2 classes.
Found 22 images belonging to 2 classes.


In [133]:
# Creamos nuestro nuevo modelo con las capas convolucionales entrenadas
for layer in modelo_convolucion.layers:
    layer.trainable = False

input_layer = modelo_convolucion.output  # Usamos la salida de nuestro modelos
x = layers.Dense(100, activation='relu')(input_layer)
x = layers.Dropout(0.3)(x)
x = layers.Dense(150, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(50, activation='relu')(x)
output_layer = layers.Dense(1, activation='sigmoid')(x)
#juntamos nuestros modelos
modelo_completo = models.Model(inputs=modelo_convolucion.input, outputs=output_layer)

checkpoint = ModelCheckpoint('Best_model_final1.hfd5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
modelo_completo.summary()

modelo_completo.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

Model: "model_29"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 178, 218, 3)]        0         []                            
                                                                                                  
 conv2d_43 (Conv2D)          (None, 177, 217, 70)         910       ['input_9[0][0]']             
                                                                                                  
 activation_44 (Activation)  (None, 177, 217, 70)         0         ['conv2d_43[0][0]']           
                                                                                                  
 activation_45 (Activation)  (None, 177, 217, 70)         0         ['activation_44[0][0]']       
                                                                                           

In [134]:
# Entrenamos nuestro modelo
modelo_completo.fit(      Train,
                steps_per_epoch=15,
                epochs=20,
                validation_data=val,
                verbose=1,
                validation_steps=5,
                callbacks=[checkpoint]
                )


Epoch 1/20
15/15 [==============================] - ETA: 0s - loss: 0.9034 - accuracy: 0.5600
Epoch 1: val_loss improved from inf to 0.95410, saving model to Best_model_final1.hfd5
15/15 [==============================] - 7s 365ms/step - loss: 0.9034 - accuracy: 0.5600 - val_loss: 0.9541 - val_accuracy: 0.5333
Epoch 2/20
15/15 [==============================] - ETA: 0s - loss: 0.7427 - accuracy: 0.6486
Epoch 2: val_loss improved from 0.95410 to 0.70065, saving model to Best_model_final1.hfd5
15/15 [==============================] - 6s 392ms/step - loss: 0.7427 - accuracy: 0.6486 - val_loss: 0.7006 - val_accuracy: 0.6000
Epoch 3/20
15/15 [==============================] - ETA: 0s - loss: 0.6284 - accuracy: 0.7432
Epoch 3: val_loss improved from 0.70065 to 0.37535, saving model to Best_model_final1.hfd5
15/15 [==============================] - 5s 346ms/step - loss: 0.6284 - accuracy: 0.7432 - val_loss: 0.3753 - val_accuracy: 0.9333
Epoch 4/20
15/15 [==============================] - ETA:

In [135]:
#cargamos nuestro mejor modelo a comeet
experiment.log_model("Best_model_final1.hfd5", filepath)
#terminamos el experimento.
experiment.end()

COMET ERROR: Experiment.log_model() requires a file or folder
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/lject/reconcimiento-final/b7c19b7bc0d34cab8f72f94767328ccb
COMET INFO:   Uploads:
COMET INFO:     conda-environment-definition : 1
COMET INFO:     conda-info                   : 1
COMET INFO:     conda-specification          : 1
COMET INFO:     environment details          : 1
COMET INFO:     filename                     : 1
COMET INFO:     installed packages           : 1
COMET INFO:     notebook                     : 1
COMET INFO:     os packages                  : 1
COMET INFO:     source_code                  : 1
COMET INFO: 
COMET WARNING: To get all data logged 

In [68]:
prediccio=model1.predict(np.expand_dims(batch_X[0], axis=0))
print(prediccio)
print(prediccio.shape)
lss=prediccio-batch_y[0]
print(batch_y[0])
print(lss)

1/1 [==============================] - 1s 564ms/step
[[1.2269891e-11 8.1255436e-01 5.5979250e-04 3.9949033e-01 2.4707867e-03
  3.0163783e-04 6.1766972e-04 4.6819844e-03 4.3273706e-04 1.3569309e-05
  3.3508575e-05 9.9639004e-01 2.2827217e-01 6.3208407e-09 2.2224143e-01
  9.9999952e-01 2.6079074e-01 1.9722795e-03 3.4035122e-01 4.3642733e-02
  4.8715971e-02 8.7114674e-01 9.9939740e-01 9.5765471e-01]]
(1, 24)
[0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1.]
[[ 1.22698908e-11 -1.87445641e-01  5.59792505e-04 -6.00509673e-01
   2.47078668e-03  3.01637832e-04  6.17669721e-04  4.68198443e-03
   4.32737055e-04  1.35693090e-05  3.35085751e-05 -3.60995531e-03
   2.28272170e-01  6.32084074e-09  2.22241431e-01 -4.76837158e-07
  -7.39209265e-01  1.97227951e-03  3.40351224e-01  4.36427332e-02
   4.87159714e-02 -1.28853261e-01 -6.02602959e-04 -4.23452854e-02]]
